In [ ]:
import pandas as pd
import requests
import json
import csv
import time
import datetime

In [ ]:
# Function for building PushShift URLs
# as seen in https://medium.com/@RareLoot/using-pushshifts-
# api-to-extract-reddit-submissions-fb517b286563

def getPushshiftData(query, after, before):
    url = 'https://api.pushshift.io/reddit/search/comment/?q='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [ ]:
# Function to extract key data points
# as seen in https://medium.com/@RareLoot/using-pushshifts-
# api-to-extract-reddit-submissions-fb517b286563

def collectSubData(subm):
    subData = list() #list to store data points
    comment_id = subm['id']
    text = subm['body']
    try:
        flair = subm['author_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    is_submitter = subm['is_submitter']
    subreddit = subm['subreddit']
    sub_id = subm['subreddit_id']
    score = subm['score']
    parent_id = subm['parent_id']
    permalink = subm['permalink']
    
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    
    subData.append((comment_id,text,flair,author,is_submitter,subreddit,
                    sub_id,score,parent_id,permalink,created))
    subStats[comment_id] = subData

In [ ]:
# Parameters to query:
# All submissions with “face mask” in their title between 1st Jan ’20
# and 22 May ’20. 
# subCount tracks the no. of total submissions we collect. 
# subStats is the dictionary where we will store our data.


# before and after dates (Unix Timestamp)
after = "1582329600" # 04/01/2018 @ 12:00am (UTC)
before = "1590182370"  # 03/01/2018 @ 12:00am (UTC)
query = "face-mask"
subCount = 0
subStats = {}

In [ ]:
# Run code
data = getPushshiftData(query, after, before)


# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before)
    
print(len(data))

In [ ]:
# Check comments
print(str(len(subStats)) + " comments have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

In [ ]:
# Upload to csv file
def updateSubs_file():
    upload_count = 0
    location = "/Users/merle-sophie/Desktop/FTL Hackathon 052020/Data"
    filename = "_Reddit_FaceMasks.csv"
    file = location + filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
       # subData.append((comment_id,text,flair,author,is_submitter,subreddit,
       #             sub_id,score,parent_id,permalink))
        headers = ["Comment ID","Text","Flair","Author","Is Submitter","Subreddit","Sub Id","Score","Parent Id", "Permalink", "Created Date"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " comments have been uploaded")
updateSubs_file()